# Credits
The steps from this notebook are an adption of the guides from
- http://www.grace-dev.com/python-apis/strava-api
- https://towardsdatascience.com/visualize-your-strava-data-on-an-interactive-map-with-python-92c1ce69e91d

# Preparation

The first step is to log into Strava and to create an app on https://developers.strava.com that will later be connected with the account and will get access to activity data through the Strava API.

Once the app has been created one will be forwarded to the settings API page of the new app from which the following entries should be stored in a `strava.ini` properties file:

    [Strava]
    client_id = XXXXX
    client_secret = XXXXX
    refresh_token = XXXXX

The properties will be read with Python's `configparser` module:

In [1]:
import configparser
config = configparser.ConfigParser()
config.read('strava.ini')
client_id = config['Strava']['client_id']
client_secret = config['Strava']['client_secret']
refresh_token = config['Strava']['refresh_token']

The following function connects with Strava's authentication API and returns both an `access_token`to be used for subsequent API calls for data retrieval and a `refresh_token` which is either the same one as the one passed into the fuction if the last access token has not expired or a new one if a new access token has been created.  

In [2]:
import requests
import json

# define function to get a new access token
def get_access_token(client_id, client_secret, refresh_token):
 
   oauth_url = 'https://www.strava.com/oauth/token'
 
   payload = {
       'client_id': client_id, 
       'client_secret': client_secret, 
       'refresh_token': refresh_token, 
       'grant_type': 'refresh_token', 
       'f': 'json', 
   }
 
   r = requests.post(oauth_url, data=payload, verify=False)
 
   access_token = r.json()['access_token']
   return (access_token, refresh_token)

# get new access token
(access_token, refresh_token) = get_access_token(client_id, client_secret, refresh_token)


/usr/lib/python3/dist-packages/urllib3/connectionpool.py:1015: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.strava.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


If the `refresh_token` has been renewed, the config file needs an update.

In [3]:
config['Strava']['refresh_token'] = refresh_token
with open('strava.ini', 'w') as configfile:
    config.write(configfile)

In [6]:
# Build the API url to get athlete info
athlete_url = f"https://www.strava.com/api/v3/athlete?" \
              f"access_token={access_token}"

# Get the response in json format
response = requests.get(athlete_url)
athlete = response.json()

print(athlete['firstname'])

Thomas


The result (in my case `Thomas`) shows that authentication and data request was successful.
Now it's time to have a look into recent activities:

In [5]:
# define function to get your strava data
def get_data(access_token, per_page=200, page=1):
 
   activities_url = 'https://www.strava.com/api/v3/athlete/activities'
   headers = {'Authorization': 'Bearer ' + access_token}
   params = {'per_page': per_page, 'page': page}
   
   data = requests.get(
       activities_url, 
       headers=headers, 
       params=params
   ).json()
 
   return data
# get you strava data
max_number_of_pages = 10
data = get_data(access_token)

print(data)

{'message': 'Authorization Error', 'errors': [{'resource': 'AccessToken', 'field': 'activity:read_permission', 'code': 'missing'}]}


Unfortunately, the result is far from what was to be expected:

    {'message': 'Authorization Error', 'errors': [{'resource': 'AccessToken', 'field': 'activity:read_permission', 'code': 'missing'}]}
    
The issue is that the standard authentication token from the registered Strava app does only come with basic `read` permissions which gives access to basic athlete information but not to activity data for the athletes. To obtain these enhanced access permissions, the respective user needs to opt in through a specific authentication procedure.       

In [7]:
redirect_uri = 'http://localhost/'

# Authorization URL
request_url = f'http://www.strava.com/oauth/authorize?client_id={client_id}' \
                  f'&response_type=code&redirect_uri={redirect_uri}' \
                  f'&approval_prompt=force' \
                  f'&scope=profile:read_all,activity:read_all'

# User prompt showing the Authorization URL
# and asks for the code
print('Click here:', request_url)
print('Please authorize the app and copy&paste below the generated code!')
print('P.S: you can find the code in the URL')
code = input('Insert the code from the url: ')

Click here: http://www.strava.com/oauth/authorize?client_id=81837&response_type=code&redirect_uri=http://localhost/&approval_prompt=force&scope=profile:read_all,activity:read_all
Please authorize the app and copy&paste below the generated code!
P.S: you can find the code in the URL
Insert the code from the url: e4d608c9f997fea731054adf352e6ceb2e63b43b


With this `code` it is now possible to request new access tokens with sufficient permissions to read activity data.

In [8]:
# Get the access token
tokens = requests.post(url='https://www.strava.com/oauth/token',
                       data={'client_id': client_id,
                             'client_secret': client_secret,
                             'code': code,
                             'grant_type': 'authorization_code'})

#Save json response as a variable
strava_tokens = tokens.json()


In [9]:
print (strava_tokens)

{'token_type': 'Bearer', 'expires_at': 1650133614, 'expires_in': 21600, 'refresh_token': '2558fae10a7c4a83ccc3701b99042d108369a498', 'access_token': '04bfab866d8754487422cb0c8ce5b90cae240c63', 'athlete': {'id': 25808221, 'username': 'tthevis', 'resource_state': 2, 'firstname': 'Thomas', 'lastname': 'Thevis', 'bio': None, 'city': 'Berlin', 'state': 'Berlin', 'country': 'Germany', 'sex': 'M', 'premium': True, 'summit': True, 'created_at': '2017-10-21T17:48:43Z', 'updated_at': '2022-04-15T19:46:34Z', 'badge_type_id': 1, 'weight': 88.0, 'profile_medium': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/25808221/7481752/2/medium.jpg', 'profile': 'https://dgalywyr863hv.cloudfront.net/pictures/athletes/25808221/7481752/2/large.jpg', 'friend': None, 'follower': None}}


In [10]:
config['Strava']['refresh_token'] = strava_tokens['refresh_token']
with open('strava.ini', 'w') as configfile:
    config.write(configfile)

In [11]:
access_token = strava_tokens['access_token']

In [12]:
print(get_data(access_token)[0])

{'resource_state': 2, 'athlete': {'id': 25808221, 'resource_state': 1}, 'name': '45 minutes base training ⛅', 'distance': 8825.9, 'moving_time': 2953, 'elapsed_time': 2957, 'total_elevation_gain': 37.2, 'type': 'Run', 'workout_type': 0, 'id': 6987020187, 'start_date': '2022-04-15T16:48:38Z', 'start_date_local': '2022-04-15T18:48:38Z', 'timezone': '(GMT+01:00) Europe/Berlin', 'utc_offset': 7200.0, 'location_city': None, 'location_state': None, 'location_country': 'Germany', 'achievement_count': 0, 'kudos_count': 9, 'comment_count': 0, 'athlete_count': 1, 'photo_count': 0, 'map': {'id': 'a6987020187', 'summary_polyline': 'qih_IuaipAC\\DpA@dBKlBWdBBp@KTQv@Dj@Nt@n@dBVf@Pp@Vr@Ln@Ph@r@zAJb@`@j@r@nBXn@Rn@Zb@hApBb@\\j@BtBj@NF\\UPCn@?Z\\PB`@Av@Qd@HXNVRLZzApAVHXV`Bz@v@p@j@h@NTz@v@hBnBx@hAnAtAXd@l@^`@b@f@NDdANv@H~@Dr@DNBADFPp@Xl@Xh@j@xA^t@VX`AL@Bd@jBPbAL~@Nt@`@jADpATlCRlEBdBh@dKLdE?tAJnB`@xDnAhFDr@DjBE`@Lp@VfBDdC@rAEtBG|@I`@Cx@Gh@m@fBKt@Wb@Sj@Mx@c@zAKb@Wh@ONq@X}@YICACS@Y^EJk@p@U^Qb@KjA_@Pc@HO\\A~

Now it is possible to fetch all activity data as a sequence of paginated `get_data()` calls and add results to a list.

In [14]:
max_number_of_pages = 10
data = list()
for page_number in range(1, max_number_of_pages + 1):
    page_data = get_data(access_token, page=page_number)
    if page_data == []:
        break
    data.append(page_data)

In [16]:
# data dictionaries
data_dictionaries = []
for page in data:
    data_dictionaries.extend(page)
# print number of activities
print('Number of activities downloaded: {}'.format(len(data_dictionaries)))

Number of activities downloaded: 812


The activities dictionary can now be persisted in a local file.

In [20]:
with open('strava_activities.txt', 'w') as activities_file: 
     activities_file.write(json.dumps(data_dictionaries))